In [51]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import copy
import pickle
import numpy as np
import pandas as pd
from matplotlib import cm
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import aplusml
import stemi
from typing import List, Optional, Dict

# Workflows
PATH_TO_YAML: str = '../workflows/stemi.yaml'
PATH_TO_PATIENT_PROPERTIES: str = '../ignore/secure/stemi/stemi.csv'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
simulation: aplusml.Simulation = aplusml.load_simulation(PATH_TO_YAML, PATH_TO_PATIENT_PROPERTIES)
simulation.draw_workflow_diagram(figsize=(50,50), path_to_file='../workflows/visualizations/STEMI Workflow.pdf')

# Load Patients + Model Predictions

First, we'll load the CSV containing our patients from `stemi.csv`.

Next, we initialize a `Patient` object for each row of our CSV, and store these objects in a list called `all_patients`.

Finally, we add each patient's properties to their `Patient` object.

In [41]:
df_patients = pd.read_csv(PATH_TO_PATIENT_PROPERTIES)
# Note: `threshold` is the same for all patients -- If model pred >= threshold, ACS is predicted
threshold = df_patients['Probability threshold below which ACS is predicted (same for all rows)'].iloc[0]
# Drop columns
df_patients.drop(df_patients.columns[0], axis=1, inplace=True)
df_patients.drop(columns=['Patient_Enc_CNS_ID', 
                          'MRN', 
                          'Probability threshold below which ACS is predicted (same for all rows)', 
                          '1 = ACS predicted;  0 = ACS not predicted',
                          'ECG_within10min',
                          'ECG1_DTS',], 
                 inplace=True)
df_patients.rename({ 'Predicted probability of having ACS (using model D)' : 'y_hat', 
                    'finaldx' : 'y',
                    'ArrivalDTS' : 'start', }, axis=1, inplace=True)
df_patients['start'] = pd.to_datetime(df_patients['start'], format='%m/%d/%y %H:%M')

# Only count "STEMI" event as positive event
df_patients['is_stemi'] = df_patients['y'] == 'STEMI'

# Assume nurse screening is always accurate
df_patients['is_screen_positive'] = df_patients['is_stemi']

# If no ECG is done, set waiting time to 0
df_patients['ArrivalDTStoECG1_min'] = df_patients['ArrivalDTStoECG1_min'].fillna(0)

f"Shape of df_patients: {df_patients.shape}"

'Shape of df_patients: (279132, 6)'

In [42]:
df_patients

,start,ArrivalDTStoECG1_min,y,y_hat,is_stemi,is_screen_positive
0,2015-01-01 00:01:00,2.0,OTHER,0.007169,False,False
1,2015-01-01 00:13:00,106.0,OTHER,0.002135,False,False
2,2015-01-01 00:19:00,0.0,OTHER,0.000644,False,False
3,2015-01-01 00:32:00,6.0,OTHER,0.009935,False,False
4,2015-01-01 00:53:00,2.0,OTHER,0.013334,False,False
...,...,...,...,...,...,...
279127,2019-12-31 23:11:00,16.0,OTHER,0.006552,False,False
279128,2019-12-31 23:12:00,0.0,OTHER,0.000270,False,False
279129,2019-12-31 23:40:00,0.0,OTHER,0.001635,False,False
279130,2019-12-31 23:47:00,0.0,OTHER,0.005075,False,False


In [43]:
# Get one month's worth of patients
df_week = df_patients[(pd.to_datetime('2019-12-31').date() >= df_patients['start'].dt.date) & (df_patients['start'].dt.date >= pd.to_datetime('2019-12-24').date())].copy()
df_month = df_patients[(pd.to_datetime('2019-12-31').date() >= df_patients['start'].dt.date) & (df_patients['start'].dt.date >= pd.to_datetime('2019-12-01').date())].copy()
print(df_month['y'].value_counts())
print(df_week['y'].value_counts())
print(df_patients['y'].value_counts())

OTHER      4703
NSTEMI       18
STEMI         2
UANGINA       1
Name: y, dtype: int64
OTHER     1259
NSTEMI       7
STEMI        1
Name: y, dtype: int64
OTHER      277735
NSTEMI        978
STEMI         225
UANGINA       194
Name: y, dtype: int64


In [44]:
# Enforce a specific prevalence of PAD
PREVALENCE: float = 0.01 # enforce a specific prevalence of PAD

# calc target number of positives rows
stemi_target = int(df_patients.shape[0] * PREVALENCE)
print(f"stemi_target: {stemi_target} out of {df_patients.shape[0]} rows")

# Upsample positive rows
df_resampled = pd.concat([
    df_patients[df_patients['y'] == 'STEMI'].sample(n=stemi_target, replace=True),
    df_patients.drop(df_patients[df_patients['y'] != 'STEMI'].sample(n=stemi_target).index),
], axis=0)

# Write df_patients to .csv
df_patients.to_csv('../ignore/secure/stemi/stemi_new.csv', index=False)
df_resampled.to_csv(f'../ignore/secure/stemi/stemi_new_prevalence_{PREVALENCE}.csv', index=False)

stemi_target: 2791 out of 279132 rows


In [46]:
simulation.metadata['path_to_properties'] = f'../ignore/secure/stemi/stemi_new_prevalence_{PREVALENCE}.csv'
MEAN_ED_VISITS_PER_DAY: int = 170
NUM_DAYS: int = 100
all_patients = stemi.generate_patient_list(simulation, mean_admits_per_day=MEAN_ED_VISITS_PER_DAY, num_days=NUM_DAYS)
all_patients[0].properties

{'y': 'OTHER',
 'y_hat': 0.003554497,
 'is_stemi': False,
 'ArrivalDTStoECG1_min': 4.0,
 'is_screen_positive': False}

In [50]:
# confirm prevalence
prevalence = 0
for x in all_patients:
    if x.properties['is_stemi']:
        prevalence += 1
prevalence /= len(all_patients)
print(f"prevalence: {prevalence} | total patients: {len(all_patients)}")

prevalence: 0.009683098591549295 | total patients: 17040


# Capacity Analysis: Physician who interprets ECG

In [52]:
THRESHOLDS = [ 0, 0.001, 0.002, 0.003, 0.004065758, 0.005, 0.01, 0.1,]
MODELS = [ 'dl', ]
PATH_TO_OUTPUT_FOLDER = '../ignore/secure/stemi/output'
os.makedirs(PATH_TO_OUTPUT_FOLDER, exist_ok=True)

In [53]:
def show_ribbon_plots(model_2_result, label_sort_order, label_title):
    p1 = aplusml.plot.plot_mean_utility_v_threshold('Deep Learning', model_2_result['dl'],
                                                    label_sort_order=label_sort_order,
                                                    label_title=label_title)
    print(p1)

def show_dodged_bar_mean_utility_plot(title: str, df_, label_sort_order, label_title, is_percent_of_optimistic):
    p = aplusml.plot.plot_dodged_bar_mean_utilities(title,
                                                    df_,
                                                    label_sort_order=label_sort_order,
                                                    color_sort_order=[
                                                        'none', 'all', 'dl', 
                                                    ],
                                                    color_names=[
                                                        'Treat\nNone', 'Deep\nLearning'
                                                    ],
                                                    is_percent_of_optimistic=is_percent_of_optimistic,
                                                    x_label=label_title)
    print(p)

def show_line_mean_utility_plot(title: str, df_, label_sort_order, label_title, is_percent_of_optimistic):
    df_['group'] = df_['color']
    p = aplusml.plot.plot_line_mean_utilities(title,
                                                df_,
                                                group_sort_order=None,
                                                shape_sort_order=None,
                                                color_title='',
                                                shape_title='',
                                                label_sort_order=label_sort_order,
                                                groups_to_drop=['none', ],
                                                color_sort_order=['all', 'dl', ],
                                                color_names=[
                                                    'Treat\nAll', 'Deep\nLearning'
                                                ],
                                                is_percent_of_optimistic=is_percent_of_optimistic,
                                                x_label=label_title)

    print(p)

def show_plots(title, model_2_result, df_, label_sort_order, label_title,
                is_show_ribbon_plot: bool = True,
                is_show_dodged_bar_plot: bool = True,
                is_show_line_plot: bool = True): 
    
    # Ribbon plot
    if is_show_ribbon_plot:
        show_ribbon_plots(model_2_result, label_sort_order, label_title)

    # Dodged bar mean utility plot
    if is_show_dodged_bar_plot:
        show_dodged_bar_mean_utility_plot(title, df_, label_sort_order, label_title, is_percent_of_optimistic=1)
    
    # Line mean utility plot
    if is_show_line_plot:
        show_line_mean_utility_plot(title, df_, label_sort_order, label_title, is_percent_of_optimistic=1)


In [ ]:
def generate_pkl_path(intro: str, patients: List[aplusml.Patient], values: list) -> str:
    pkl_name: str = (intro +
                     "-".join([str(x) for x in values]) + "_" +
                     str(len(patients)) + "_" +
                     ".pkl")
    pkl_path: str = os.path.join(PATH_TO_OUTPUT_FOLDER, pkl_name)
    os.makedirs(os.path.dirname(pkl_path), exist_ok=True)
    return pkl_path

def load_cached_pkl(pkl_path: str):
    with open(pkl_path, 'rb') as fd:
        results = pickle.load(fd)
    model_2_result, baseline_2_result = results['model'], results['baseline']
    return model_2_result, baseline_2_result

def generate_results(intro: str, patients: List[aplusml.Patient], values: list):
    pkl_path: str = generate_pkl_path(intro, patients, values)
    if os.path.exists(pkl_path):
        print(f"Loading file from {pkl_path}")
        model_2_result, baseline_2_result = load_cached_pkl(pkl_path)
    else:
        print(f"Didn't find {pkl_path}, recreating from scratch")
        labels = [f"{x}" for x in values]
        model_2_result, baseline_2_result = stemi.run_test(patients,
                                                         labels,
                                                         MODELS,
                                                         THRESHOLDS,
                                                         PATH_TO_PATIENT_PROPERTIES)
        with open(pkl_path, 'wb') as fd:
            pickle.dump({
                'model': model_2_result,
                'baseline': baseline_2_result,
            }, fd)
    return model_2_result, baseline_2_result

def simulate_workflow(patients: List[aplusml.Patient],
                            values: list,
                            path_to_yaml: str = None,
                            is_show_ribbon_plot: bool = True,
                            is_show_dodged_bar_plot: bool = True,
                            is_show_line_plot: bool = True):
    values += [ 1e5, ] # 1e5 represents the optimistic case
    
    model_2_result, baseline_2_result = generate_results('nurse_', patients, values, path_to_yaml)

    # Plotting constants
    df_ = stemi.plot_helper(model_2_result, {
        'all': baseline_2_result['all'],
        'none': baseline_2_result['none']
    }).rename(columns={'model': 'color'})
    # Use 1e5 as the 'optimistic' case for nurse capacity
    df_ = df_[df_['label'] != 'optimistic']
    df_.loc[df_['label'] == '100000.0', 'label'] = 'optimistic'
    label_sort_order: List[str] = [ str(x) for x in df_['label'].unique() ]
    label_title: str = 'Nurse Capacity'

    for m in MODELS:
        model_2_result[m] = model_2_result[m][model_2_result[m]['label'] != 'optimistic']
        model_2_result[m].loc[model_2_result[m]['label'] == '100000.0', 'label'] = 'optimistic'
        
    title = f'Utility of STEMI workflow'
    show_plots(title, model_2_result, df_, label_sort_order, label_title,
                is_show_ribbon_plot = is_show_ribbon_plot,
                is_show_dodged_bar_plot = is_show_dodged_bar_plot,
                is_show_line_plot = is_show_line_plot)
    

In [ ]:
values = [0, 1, 2, 3, ]
simulate_workflow(copy.deepcopy(all_patients),
                        values,
                        path_to_yaml=PATH_TO_YAML,
                        is_show_dodged_bar_plot=True)